In [186]:
import math
import numpy as np

import geopandas as gpd
import pandas as pd

import pyproj
from shapely import geometry

import verde as vd
import rioxarray as rio

import matplotlib.pyplot as plt

import tqdm

from tqdm.notebook import trange

gdb = '/home/ggrl/geodatabase/'




#                                      DEFININDO FUNÇÕES PARA SCRIPT 

# Definindo Regions (W,E,S,N)
def regions(gdf):
    # Criando Region em coordenadas geograficas
    bounds = gdf.bounds 
    gdf['region'] = \
    [(left,right,bottom,top) for left,right,bottom,top in zip(bounds['minx'],bounds['maxx'],
                                                              bounds['miny'],bounds['maxy'])]
    
    # Criando Region em coordenadas Projetadas    #gdf = gdf.set_crs(32723, allow_override=True)
    gdf = gdf.to_crs("EPSG:32723")
    bounds = gdf.bounds 
    gdf['region_proj'] = \
    [(left,right,bottom,top) for left,right,bottom,top in zip(bounds['minx'],bounds['maxx'],
                                                          bounds['miny'],bounds['maxy'])]
    
    return gdf
        

# Definindo nomes da malha a partir da articulação sistematica de folhas de cartas. Construindo uma lista e definindo como uma series.
def nomeador_malha(gdf):
    df = pd.DataFrame(gdf)
    lista_grid = []
    for index, row in df.iterrows():
        row['id_folha'] = (nomeador_grid(row.region[0],row.region[1],
                                         row.region[3],row.region[2],escala=5))
        lista_grid.append(row.id_folha)

    gdf['id_folha'] = lista_grid

# Selecionador de Região
def select_area(escala,id):
    malha_cartog = importar('malha_cartog_'+escala+'_wgs84')
    malha_cartog_gdf_select = malha_cartog[malha_cartog['id_folha'].str.contains(id)]       # '.contains' não é ideal.
    malha_cartog_gdf_select = regions(malha_cartog_gdf_select)    
    
    return(malha_cartog_gdf_select)

# Importador de Litologias por escala
def importar(camada, mapa=False):
    lito =  gpd.read_file(gdb+'database.gpkg',
                        driver= 'GPKG',
                        layer= camada)
    if mapa:
        folha = lito[lito.MAPA == 'Carta geológica da folha '+mapa]
        return(folha)
    else:
        return(lito)
    
# Listando regiões das folhas cartográficas
def cartas(escala,id):
    malha_cartog_gdf_select = select_area(escala,id)
    
    
    # Apenas uma folha de carta
    if (malha_cartog_gdf_select.shape[0]) > 1:
        
        # Criando dicionário de cartas
        malha_cartog_gdf_select.set_index('id_folha',inplace=True)
        malha_cartog_gdf_select.drop(columns=['geometry'],inplace=True)
        dic_cartas = malha_cartog_gdf_select.to_dict()
        
        for n in trange(len(malha_cartog_gdf_select)):
            
            print('ok')
            
        print(f"#{len(malha_cartog_gdf_select)} folhas cartográfica em escala de {escala} selecionadas: {list(malha_cartog_gdf_select.index)}")
        
    # Mais de uma folha de carta
    if len(malha_cartog_gdf_select) == 1:
        print(f"#{len(malha_cartog_gdf_select)} folha cartográfica em escala de {escala} selecionada: {list(malha_cartog_gdf_select.index)}")
        
    
    return malha_cartog_gdf_select ,dic_cartas  

In [182]:
cartas('100k','SF23_V_D')

/root/anaconda3/envs/geologist_bot/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


  0%|          | 0/6 [00:00<?, ?it/s]

ok
ok
ok
ok
ok
ok
#6 folhas cartográfica em escala de 100k selecionadas: ['SF23_V_D_I', 'SF23_V_D_IV', 'SF23_V_D_II', 'SF23_V_D_V', 'SF23_V_D_III', 'SF23_V_D_VI']


(                                    region  \
 id_folha                                     
 SF23_V_D_I    (-46.5, -46.0, -21.5, -21.0)   
 SF23_V_D_IV   (-46.5, -46.0, -22.0, -21.5)   
 SF23_V_D_II   (-46.0, -45.5, -21.5, -21.0)   
 SF23_V_D_V    (-46.0, -45.5, -22.0, -21.5)   
 SF23_V_D_III  (-45.5, -45.0, -21.5, -21.0)   
 SF23_V_D_VI   (-45.5, -45.0, -22.0, -21.5)   
 
                                                     region_proj  
 id_folha                                                         
 SF23_V_D_I    (344093.4542657379, 396417.3669110873, 7621768...  
 SF23_V_D_IV   (344618.78861818794, 396775.3595709229, 756641...  
 SF23_V_D_II   (396067.23863707075, 448210.13570348773, 76221...  
 SF23_V_D_V    (396417.3669110873, 448389.1032635307, 7566835...  
 SF23_V_D_III  (448035.0999465586, 500000.0, 7622431.53520017...  
 SF23_V_D_VI   (448210.13570348773, 500000.0, 7567088.6813917...  ,
 {'region': {'SF23_V_D_I': (-46.5, -46.0, -21.5, -21.0),
   'SF23_V_D_IV': (-46.5, -4

In [184]:
#                                      DEFININDO FUNÇÃO DE SCRIPT    
def interpolar(escala,id,geof,degree=2,spacing=499,psize=100,n_splits=False,save=False,lista_canal=['CTCOR','eU','eth','MDT','KPERC'],describe_data=True,nome='Rio Paraim'):
    
    # Listando regiões das folhas cartográficas
    malha_cartog_gdf_select, dic_cartas = cartas(escala,id)

                    
    # Listando colunas do dado aerogeofísico
    print(f"Lista de atributos:  {list(geof.columns)}")
    
    # Dicionário de dados interpolados
    grids = {lista_canal[0]:(),lista_canal[1]:(),lista_canal[2]:(),lista_canal[3]:(), lista_canal[4]:()}
    
    # Dicionário validação cruzada
    scores = {lista_canal[0]:(),lista_canal[1]:(),lista_canal[2]:(), lista_canal[3]:(), lista_canal[4]:()}
    
    
    
    # Iterando entre itens da lista de folhas cartográficas
    for n in trange(1):
        for index, row in malha_cartografica.iterrows():
            print(f"# -- Início da iteração da folha: {row.id_folha} #")
            print(f"# -- Recortando dados da folha: {row.id_folha} #")

            data = geof[vd.inside((geof.X, geof.Y), region = row.region_proj)]
            coordinates = (data.X.values, data.Y.values)

            #print('# Distribuição')
            #print(f"{data['CTCOR'].describe(percentiles = [0.02, 0.25, 0.50, 0.75, 0.995])}")
            #print('# Distribuição')
            #print(f"{data['eU'].describe(percentiles = [0.02, 0.25, 0.50, 0.75, 0.995])}")


            if data.empty or len(data) < 1000:  # if data dont have values pass to next step
                print('não há dados aerogofísicos para folha cartográfica de id: '+row.id_folha)
                print(f"A folha possui:  {len(data)} pontos coletados")

            else:
                print(f"# -- Inicio da interpolação -- # ")
                print(f"com {len(data)} pontos de contagens radiométricas coletados")

                # Iterando entre os canais de interpolação
                print("pipeline")
                print(f" {chain} ")
                for n in trange(1):
                    print('fit: ')
                    for i in lista_canal:
                        chain.fit(coordinates, data[i])
                        print(i)

                        # Griding the predicted data.  
                        grid = chain.grid(spacing=psize, data_names=[i],pixel_register=True)
                        grids[i] = vd.distance_mask(coordinates, maxdist=spacing, grid= grid)
                        
                        # Adicionando grids ao dicionario
                        dic_cartas['grids']= append.grids
                        
                        # Processo de validação cruzada da biblioteca verde
                        if n_splits:
                            cv     = vd.BlockKFold(spacing=spacing,
                                        n_splits=n_splits,
                                        shuffle=True)

                            scores[i] = vd.cross_val_score(chain,
                                                    coordinates,
                                                    data[i],
                                                    cv=cv)
                            
                        # Adicionando scores ao dicionario
                        dic_cartas['scores'] = append.scores
                        
                        # Salvar os dados interpolados em formato .tif
                        if save:
                            print('salvando '+row.id_folha+' '+i)
                            #grids[i].to_netcdf(gdb+'/grids/geof_3022_'+str(psize)+'m_'+i+'_'+row.id_folha+'.nc')
                            tif_ = grids[i].rename(easting = 'x',northing='y')
                            tif_.rio.to_raster(gdb+'grids/geof_'+str(save)+'_'+str(psize)+'m_'+i+'_'+row.id_folha+'.tif')
                        
                        # Adicionando grids e score
                        dic_cartas['grids'], dic_cartas['scores'] = append.grids, append.scores
                    
                    print(f"# -- Fim da interpolação -- # {row.id_folha}'")

                    # Descrição estatisica das contagens
                    if describe_data:
                        dataframe = list()
                        for i in lista_canal:
                            df = grids[i].to_dataframe()
                            dataframe.append(df[i])
                        geof_grids = pd.concat(dataframe,axis=1, join='inner')
                        geof_grids.reset_index(inplace=True)

                        geof_grids['geometry'] =\
                             [geometry.Point(x,y) for x, y in zip(geof_grids['easting'], geof_grids['northing'])]

                        for n in trange(1):
                            print('Ajustando crs')
                            gdf = gpd.GeoDataFrame(geof_grids,crs=32723)
                            gdf = gdf.set_crs(32723, allow_override=True)
                            gdf = gdf.to_crs("EPSG:32723")
                            print(f" geof: {gdf.crs}")

                            #litologia=importar(lito,"Rio de Janeiro")
                            litologia = importar('l_100k',nome)
                            litologia.reset_index(inplace=True)
                            litologia = litologia.set_crs(32723, allow_override=True)
                            litologia = litologia.to_crs("EPSG:32723")
                            print(f" lito: {litologia.crs}")

                            print(f"# Listando de siglas de unidades litológicas do mapa {litologia['MAPA'].unique()}:       {list(litologia['SIGLA'].unique())} ") 

                            print(f"# Calculando geometria mais próxima para cada um dos {len(geof_grids)} pixels da folha {row.id_folha}")


                            geof_grids['closest_unid'] = gdf['geometry'].apply(lambda x: litologia['SIGLA'].iloc[litologia.distance(x).idxmin()])

                            print(f"# siglas de unidades litológicas presentes na folha de id {row.id_folha}:    {list(geof_grids['closest_unid'].unique())}")
                            
                            
                            
                            
                            
                            #print('# Distribuição')
                            #print(f"{geof_grids['CTCOR'].describe(percentiles = [0.02, 0.25, 0.50, 0.75, 0.995])}")

                            #print('# Distribuição')
                            #print(f"{geof_grids['eU'].describe(percentiles = [0.02, 0.25, 0.50, 0.75, 0.995])}")
                            print(f"#{row.id_folha}'# -- Fim da iteração -- #'")
                            print('__________________________________________')


        return data, grids, geof_grids




In [177]:
malha_cartog_gdf_select, dic_cartas = cartas('25k','SC23_Z_A_IV_2_N')

/root/anaconda3/envs/geologist_bot/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


  0%|          | 0/2 [00:00<?, ?it/s]

ok
ok
#2 folhas cartográfica em escala de 25k selecionadas: ['SC23_Z_A_IV_2_NW', 'SC23_Z_A_IV_2_NE']


In [178]:
malha_cartog_gdf_select

,region,region_proj
id_folha,,
SC23_Z_A_IV_2_NW,"(-44.75, -44.625, -10.625, -10.5)","(527344.9814102683, 541034.2069596786, 8825459..."
SC23_Z_A_IV_2_NE,"(-44.625, -44.5, -10.625, -10.5)","(541017.6247691072, 554712.5614995236, 8825439..."


In [179]:
dic_cartas

{'region': {'SC23_Z_A_IV_2_NW': (-44.75, -44.625, -10.625, -10.5),
  'SC23_Z_A_IV_2_NE': (-44.625, -44.5, -10.625, -10.5)},
 'region_proj': {'SC23_Z_A_IV_2_NW': (527344.9814102683,
   541034.2069596786,
   8825459.237341102,
   8839294.029712576),
  'SC23_Z_A_IV_2_NE': (541017.6247691072,
   554712.5614995236,
   8825439.98706358,
   8839280.434275124)}}

In [174]:
#malha_cartog_gdf_select.set_index('id_folha',inplace=True)

#len(list(malha_cartog_gdf_select.index))